El siguiente desafío busca introducir el problema fundamental de la Ciencia de Datos que es el
overfitting desde un ángulo distinto, presentando los fenómenos del efecto del tamaño de la
muestra, el conocido problema de las múltiples comparaciones y “la maldición del ganador”.

# Definición del problema

## 0) Caso fallido: "una jugadora se quiere pasar de lista"

Una es super talentosa se llama Diana Taurasi, nosotros sabemos que tiene un “índice de enceste”
del 0.85, es decir cada vez que hace un tiro libre su probabilidad de encestar es del 85%. Si hace
100 tiros libres, en promedio encesta 85 veces, a veces puede ser más, a veces menos, pero en
promedio encesta 85 de 100.

La otra es un auténtico desastre, a quien llamaremos Miss X, y su índice de enceste es de 0.10.
Esto quiere decir que si hace 100 tiros libres, en promedio encesta 10.
La cazatalentos NO tiene forma de conocer los valores de 0.85 y 0.10, solo puede hacerlas tirar y
contar aciertos.

La cazatalentos llega al gimnasio, y solo se encuentra Diana, con lo cual le indica que comience a
tirar. Al tiro 10, Diana lleva 8 encestes y dos no-enceste.


En ese momento, llega con un andar cansino Miss X y las interrumpe : ¿cuánto va Diana ? La
cazatalentos le responde 8 de 10. Miss X solicita permiso para tirar, y acierta ese tiro.
Ahí mismo dice : “paren las rotativas, yo soy la clara ganadora de este torneo, mi índice de acierto
es del 100% y el de Diana es del 80% , la matemática es clara”.

La cazatalentos monta en cólera y le dice : pues usted joven solo ha realizado un tiro, ¿Si lo
hubiera errado, aceptaría decir entonces que su índice de acierto es del 0%? Ciertamente no.
Pues ahora, usted deberá seguir tirando hasta completar sus 10 tiros libres, y recién ahí
compararé.

## 1) Primer caso, sin overfitting

La cazatalentos tiene el siguiente algoritmo, hace que Diana Taurasi y Miss X hagan 10 tiros libres
cada una, y elige a la ganadora de esos 10 tiros libres.

¿Garantiza este método que por lo menos el 99% de las veces elige a Diana Taurasi?
¿La desastrosa Miss X puede llegar a encestar los 10 tiros libres? Efectivamente sí existe esa
posibilidad, pero con una muy baja probabilidad, la probabilidad es de 0.10 ^10 = 1e-10

El símbolo de ^ significa "elevado a la potencia de"
¿Cual es la probabilidad que Miss X tire una sola vez y acierte este tiro? Simplemente 0.1 , una en
diez, con lo cual ha tenido mucha suerte en su primer tiro, pero no la va a poder mantener !

¿La adolescente Diana Taurasi puede llegar a encestar los 10 tiros libres? Efectivamente, su
probabilidad es 0.85 ^10 = 0.1968744

En el lenguaje Python la instrucción random.random(10) genera un vector con 10 números
aleatorios con distribución uniforme en el intervalo [0,1], por ejemplo

In [1]:
import numpy as np
np.random.random(10)

array([0.26950279, 0.15320624, 0.7695055 , 0.33183013, 0.51932396,
       0.0030203 , 0.6332249 , 0.35497789, 0.03610054, 0.4688969 ])

La instrucción sum( np.random.random(10) < 0.85 ) calcula cuántos de los 10 valores son
menores a 0.85, es decir, cuantos encestes hizo Michael Jordan en esos 10 tiros libres.
Ahora simulamos 10000 veces la estrategia de tirar 10 tiros libres y quedarse con el ganador.

Ahora simulamos 10000 veces la estrategia de tirar 10 tiros libres y quedarse con el ganador.

In [7]:
missX_ganadora = 0

for i in range(10000):
    aciertos_diana = sum(np.random.random(10) < 0.85)
    aciertos_missX = sum(np.random.random(10) < 0.10)
    if (aciertos_missX > aciertos_diana) :
        missX_ganadora = missX_ganadora + 1

print(missX_ganadora)

0


Lo que da la cantidad de 0, o sea que en 10000 veces, siempre ganó Diana Taurasi, con lo cual la
estrategia de quedarse con el mejor de 10 tiros ha funcionado, para este obvio caso donde hay
apenas dos jugadoras y hay una diferencia abismal entre ellas.

En este caso NO aparece el overfitting por ningún lado

## 2) Segundo Caso, aparece el overfitting

La cazatalentos llega ahora a un pueblo donde hay 100 jugadoras las que son mucho más parejas
entre sí desde el punto de vista del ratio de enceste.

Hay una jugadora, que llamaremos jugadora 1 que tiene un “índice de enceste” de 0.70
Las 99 jugadoras restantes que llamaremos “el pelotón” tienen los índices de enceste { 0.501,
0.0502, 0.503, …, 0.599 }
Es decir la jugadora 1 tiene 0.70 y la mejor jugadora del pelotón tiene 0.599, hay un poco más de
0.10 de diferencia. La diferencia de 0.10 es significativa en basket.

¿Qué sucede con el algoritmo de la cazatalentos de hacer tirar 10 tiros libres a cada una de las
jugadoras y elegir la que más encestes logró ?

In [8]:
#Pongo mi primer semilla
np.random.seed(279511)

# calcula cuantos encestes logra un jugadora con indice de enceste prob
# haciendo qyt tiros libres

def ftirar(prob, qty):
  return sum(np.random.rand(qty) < prob)



# defino los jugadoras
mejor = 0.7
peloton = np.array(range(501, 600)) / 1000
jugadoras = np.append(mejor, peloton)

# veo que tiene el vector
jugadoras

# vectorizo la funcion  ftirar
vec_ftirar = np.vectorize(ftirar)

# hago que las 100 jugadoras tiren 10 veces cada una
vec_ftirar(jugadoras, 10)

primera_ganadora = 0

for i in range(10000): # diez mil experimentos
  vaciertos = vec_ftirar(jugadoras, 10) # 10 tiros libres cada jugadora
  mejor_ronda = np.argmax(vaciertos)
  if mejor_ronda == 0:
    primera_ganadora += 1




print(primera_ganadora)

1283


Aquí ya pasa algo asombroso, el overfitting en todo su esplendor.

Si hago tirar 10 tiros libres a cada una de las 100 jugadoras, apenas 1450 veces de las 10000 este
método devuelve a la verdadera mejor, que es la jugadora con un índice de enceste de 0.70

Intuitivamente, lo que está sucediendo es que la inmensa mayoría de las veces, una de las 99
jugadoras del pelotón tiene mucha suerte y supera a la jugadora de 0.70, con lo cual se elige a una
del pelotón y no a la verdadera mejor ! El tener 99 jugadoras en el pelotón hace que al ser tantas
hay varias que tienen mala suerte, pero también hay varias con muy buena suerte, y la que tuvo
más suerte supera a la verdadera mejor !